The data is loaded in and preprocessed almost the same as last time. The only difference is the first column does not need to be divided by 1000 as we are going to normalize the data column-wise instead of row-wise.

In [151]:
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=5, suppress=True)

dev = qml.device("default.qubit", wires=4)

train_set = np.loadtxt(open("mock_train_set.csv"), delimiter=",")
test_set = np.loadtxt(open("mock_test_set.csv"), delimiter=",")

def preprocessing(data):
    data[1:,1] = np.log10(data[1:,1])
    data[1:,2] = np.log10(data[1:,2])
    data[1:,3] = np.sin(data[1:,3]*np.pi/180)
    data[1:,4] = 2 * data[1:,4] - 1
    return data

train_set = preprocessing(train_set)
test_set = preprocessing(test_set)

Next we normalize the data to be in the interval [-π,π]. Unlike the amplitude encoding, we normalize the data column-wise instead of row-wise, as we are not normalizing the statevector, but rather ensuring our data falls within an interval where the rotations are uniquely defined.

In [152]:
# Normalizes data between -π and π
def get_norms(x):
    n_rows, n_cols = x.shape
    maxes = np.max(np.abs(x), axis=0)
    params = x / np.tile(maxes, (n_rows, 1))
    params[:,0:3] *= np.pi
    return params

print("First 3 rows of raw train_set data:\n", train_set[1:,:][0:3])

train_set = get_norms(train_set[1:])
test_set = get_norms(test_set[1:])

print("First 3 rows of normalized train_set data:\n", train_set[:][0:3])

features = train_set[:,0:4]

First 3 rows of raw train_set data:
 [[2789.26       3.         1.         0.34202   -1.     ]
 [4040.01       6.         0.         0.01745    1.     ]
 [2931.2        4.         4.         0.64279    1.     ]]
First 3 rows of normalized train_set data:
 [[ 1.75408  1.5708   0.5236   0.34202 -1.     ]
 [ 2.54064  3.14159  0.       0.01745  1.     ]
 [ 1.84335  2.0944   2.0944   0.64279  1.     ]]


Our circuit for the angle encoding is considerably simpler than for the amplitude encoding. It entails just a single rotation gate on each qubit:

In [144]:
# Prepares state with rotation gates
def state_preparation(a):
    qml.RY(a[0], wires=0)
    qml.RY(a[1], wires=1)
    qml.RY(a[2], wires=2)
    qml.RY(a[3], wires=3)

The functions defining our full circuit, our weighted layers, our classifier, loss, accuracy and cost can all be defined as they were for the amplitude encoding:

In [145]:
@qml.qnode(dev)
def circuit(weights, params):
    state_preparation(params)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[0, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[1, 0])

def variational_classifier(weights, bias, params):
    return circuit(weights, params) + bias

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2
    loss /= len(labels)
    return loss

def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)

The data is split into 80% training and 20% validation:

In [146]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.8 * num_data)
index = np.random.permutation(range(num_data))
feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

We now need 4 qubits and choose to double the number of layers:

In [147]:
num_qubits = 4
num_layers = 12

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

The Adam optimizer is used and the circuit is trained over 60 steps. As we saw earlier, the cost is seen to steadily decease, and the accuracy of the training and validation sets increases.

In [150]:
from pennylane.optimize import AdamOptimizer

opt = AdamOptimizer()
batch_size = 10

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(60):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    )

Iter:     1 | Cost: 1.4659866 | Acc train: 0.5291667 | Acc validation: 0.5666667 
Iter:     2 | Cost: 1.4327932 | Acc train: 0.5250000 | Acc validation: 0.5500000 
Iter:     3 | Cost: 1.4003558 | Acc train: 0.5375000 | Acc validation: 0.5333333 
Iter:     4 | Cost: 1.3706410 | Acc train: 0.5416667 | Acc validation: 0.5500000 
Iter:     5 | Cost: 1.3425421 | Acc train: 0.5375000 | Acc validation: 0.5500000 
Iter:     6 | Cost: 1.3147193 | Acc train: 0.5375000 | Acc validation: 0.5333333 
Iter:     7 | Cost: 1.2859176 | Acc train: 0.5458333 | Acc validation: 0.5166667 
Iter:     8 | Cost: 1.2559611 | Acc train: 0.5583333 | Acc validation: 0.5166667 
Iter:     9 | Cost: 1.2263932 | Acc train: 0.5625000 | Acc validation: 0.5166667 
Iter:    10 | Cost: 1.1967621 | Acc train: 0.5666667 | Acc validation: 0.5166667 
Iter:    11 | Cost: 1.1691493 | Acc train: 0.5833333 | Acc validation: 0.5166667 
Iter:    12 | Cost: 1.1414791 | Acc train: 0.6000000 | Acc validation: 0.5500000 
Iter:    13 | Co

We calculate the accuracy of our test set:

In [149]:
# process our test data like we did our train data

test_features = test_set[:, 0:4]
print("First 3 features:\n", test_features[0:3])

Y_test = test_set[:, -1]

predictions_test = [np.sign(variational_classifier(weights, bias, f)) for f in test_features]
acc_test = accuracy(Y_test, predictions_test)
print("\nTesting accuracy: {}".format(acc_test))

First 3 features:
 [[1.91088 2.0944  2.0944  0.96593]
 [2.18279 0.      1.0472  1.     ]
 [2.48824 0.      0.      0.08716]]

Testing accuracy: 0.9416666666666667


The accuracy for the test set is found to be ~ 94%, demonstrating this model has good predictive power.

Noting that the third and fourth wires were not utilized with the former layout, I added two more rotations:

In [154]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 2], W[1, 0], wires=1)
    qml.Rot(W[0, 1], W[0, 1], W[1, 2], wires=2)
    qml.Rot(W[1, 0], W[0, 1], W[1, 0], wires=3)
    qml.CNOT(wires=[1, 0])
    
num_qubits = 4
num_layers = 12

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = AdamOptimizer()
batch_size = 10

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(60):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    )
    

predictions_test = [np.sign(variational_classifier(weights, bias, f)) for f in test_features]
acc_test = accuracy(Y_test, predictions_test)
print("\nTesting accuracy: {}".format(acc_test))

Iter:     1 | Cost: 1.4696014 | Acc train: 0.5291667 | Acc validation: 0.5666667 
Iter:     2 | Cost: 1.4360756 | Acc train: 0.5250000 | Acc validation: 0.5500000 
Iter:     3 | Cost: 1.4055779 | Acc train: 0.5250000 | Acc validation: 0.5333333 
Iter:     4 | Cost: 1.3766791 | Acc train: 0.5375000 | Acc validation: 0.5500000 
Iter:     5 | Cost: 1.3562691 | Acc train: 0.5375000 | Acc validation: 0.5500000 
Iter:     6 | Cost: 1.3333253 | Acc train: 0.5458333 | Acc validation: 0.5333333 
Iter:     7 | Cost: 1.3114533 | Acc train: 0.5375000 | Acc validation: 0.5333333 
Iter:     8 | Cost: 1.2877446 | Acc train: 0.5416667 | Acc validation: 0.5333333 
Iter:     9 | Cost: 1.2654644 | Acc train: 0.5416667 | Acc validation: 0.5166667 
Iter:    10 | Cost: 1.2424248 | Acc train: 0.5500000 | Acc validation: 0.5166667 
Iter:    11 | Cost: 1.2193529 | Acc train: 0.5500000 | Acc validation: 0.5166667 
Iter:    12 | Cost: 1.1941335 | Acc train: 0.5541667 | Acc validation: 0.5166667 
Iter:    13 | Co

This result was very similar to the last. In the next run, I will add more gates to the layers:

In [155]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 2], W[1, 0], wires=1)
    qml.Rot(W[0, 1], W[0, 1], W[1, 2], wires=2)
    qml.Rot(W[1, 0], W[0, 1], W[1, 0], wires=3)
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[3, 2])
    qml.Rot(W[0, 1], W[1, 1], W[1, 2], wires=0)
    qml.Rot(W[1, 0], W[2, 2], W[1, 2], wires=1)
    qml.Rot(W[1, 1], W[0, 0], W[0, 2], wires=2)
    qml.Rot(W[1, 2], W[1, 0], W[2, 0], wires=3)
    qml.CNOT(wires=[3, 1])
    qml.CNOT(wires=[2, 0])
    
num_qubits = 4
num_layers = 12

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = AdamOptimizer()
batch_size = 10

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(60):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    )
    

predictions_test = [np.sign(variational_classifier(weights, bias, f)) for f in test_features]
acc_test = accuracy(Y_test, predictions_test)
print("\nTesting accuracy: {}".format(acc_test))

Iter:     1 | Cost: 1.4201673 | Acc train: 0.5166667 | Acc validation: 0.6000000 
Iter:     2 | Cost: 1.3453872 | Acc train: 0.5291667 | Acc validation: 0.6000000 
Iter:     3 | Cost: 1.2808992 | Acc train: 0.5250000 | Acc validation: 0.6166667 
Iter:     4 | Cost: 1.2172721 | Acc train: 0.5333333 | Acc validation: 0.6333333 
Iter:     5 | Cost: 1.1505229 | Acc train: 0.5458333 | Acc validation: 0.6666667 
Iter:     6 | Cost: 1.0868215 | Acc train: 0.5666667 | Acc validation: 0.6833333 
Iter:     7 | Cost: 1.0243271 | Acc train: 0.5708333 | Acc validation: 0.6666667 
Iter:     8 | Cost: 0.9650377 | Acc train: 0.5875000 | Acc validation: 0.6833333 
Iter:     9 | Cost: 0.9068989 | Acc train: 0.6041667 | Acc validation: 0.7166667 
Iter:    10 | Cost: 0.8505675 | Acc train: 0.6375000 | Acc validation: 0.7166667 
Iter:    11 | Cost: 0.8036229 | Acc train: 0.6666667 | Acc validation: 0.7333333 
Iter:    12 | Cost: 0.7685566 | Acc train: 0.7166667 | Acc validation: 0.7500000 
Iter:    13 | Co

The cost was lower than in the last run, and the accuracy on both the training and validation sets was higher. The accuracy on the test set was higher as well, at about 98%. The angle encoding appeared to benefit from this deeper circuit. One downside of this deeper circuit is that training took a longer amount of time to finish.